# GPU Computing - Exercise 2
## Eetu Knutars
## 21.1.2025

### Task 1 + 2
Implement vector differentiation similar to NumPy diff function. Extend the kernel to use shared memory.

Importing libraries

In [1]:
import cupy as cp
import numpy as np

Defining the kernel

In [2]:
# Define the CUDA kernel for differentiation using shared memory
diff_kernel = cp.RawKernel(r'''extern "C"
__global__ void diff_shared(const double* input, double* output, int array_size) {
    extern __shared__ double shared_input[];

    int thread_index = threadIdx.x;
    int global_index = blockIdx.x * blockDim.x + thread_index;

    // Load elements into shared memory
    if (global_index < array_size) {
        shared_input[thread_index] = input[global_index];
    }

    // Load the next element for the last thread in the block (if within bounds)
    if (thread_index == blockDim.x - 1 && global_index < array_size - 1) {
        shared_input[thread_index + 1] = input[global_index + 1];
    }
    __syncthreads();

    // Compute differences using shared memory
    if (global_index < array_size - 1) {
        output[global_index] = shared_input[thread_index + 1] - shared_input[thread_index];
    }
}''', 'diff_shared')

Generate the vectors and call the kernel to calculate difference

In [3]:
def diff_cupy_shared(input_array):
    """
    Perform differentiation similar to NumPy's diff function using CuPy and shared memory.

    Parameters:
        input_array (cp.ndarray): Input 1D array.

    Returns:
        cp.ndarray: Output array of differences.
    """
    n = input_array.size
    if n < 2:
        raise ValueError("Input array must have at least two elements.")

    # Allocate memory for the output array
    output_array = cp.zeros(n - 1, dtype=cp.float64)

    # Define block and grid sizes
    threads_per_block = 256
    blocks_per_grid = (n + threads_per_block - 1) // threads_per_block

    # Shared memory size (threads_per_block * sizeof(double))
    shared_mem_size = threads_per_block * cp.float64().itemsize

    # Launch the kernel
    diff_kernel((blocks_per_grid,), (threads_per_block,),
                (input_array, output_array, n),
                shared_mem=shared_mem_size)

    return output_array

input = cp.array([1., 2., 0., 4., 5.])
if (np.allclose(diff_cupy_shared(input), np.diff(input), 0.001, 0.001)):
  print("They are the same!")
else:
  print("Something must have gone wrong.:(")

They are the same!


# Task 3
Extend the matrix-matrix multiplication kernel from lectures to use shared memory.

Import libraries

In [4]:
import cupy as cp
import math
import numpy as np

Defining the kernel

In [5]:
our_kernel = cp.RawKernel(r''' extern "C"
#define TILE_DIM 32

__global__ void matrix_multiplication_shared_memory(const double* A, const double* B, double* C, int N, int M, int K)
{

	  // Allocate the sub-matrices to the shared memory. Note two-dim indexing.
    __shared__ double sub_A[TILE_DIM][TILE_DIM];
    __shared__ double sub_B[TILE_DIM][TILE_DIM];

    const int tx = threadIdx.x;
    const int ty = threadIdx.y;

    const int bx = blockIdx.x;
    const int by = blockIdx.y;

    // Each block gets it own TILE_DIM sized slot in x and y directions.
    const int row = by * TILE_DIM + ty;
    const int col = bx * TILE_DIM + tx;

    double result = 0.0;

    for(int i = 0; i <  M  / TILE_DIM; i++)
        {

        // Iterate over the tile dimension to copy the data.
        sub_A[ty][tx] = A[(i * TILE_DIM + tx) + M * row];
        sub_B[ty][tx] = B[(i * TILE_DIM + ty) * K + col];

		    // Make sure that all threads have completed the memory transaction.
        __syncthreads();

        // Multiply the matrix elements inside the tile and add them to the result.
        for (int j = 0; j < TILE_DIM; j++)
            {
            result += sub_A[ty][j] * sub_B[j][tx];
            }

		    // Make sure that all of the threads have finished the calculation
        __syncthreads();
        }

    // Write back to the global memory. Using the global index.
    int C_index = K * (by * blockDim.y + ty) + (bx * blockDim.x + tx);
    C[C_index] = result;
}
''', 'matrix_multiplication_shared_memory')

Generating the input matrices and calling the kernel

In [6]:
# Define matrix dimensions
A_n_rows = 1024
A_n_cols = 512
B_n_rows = 512
B_n_cols = 256
C_n_rows = A_n_rows
C_n_cols = B_n_cols

value_type = cp.float64

# Define the block and grid sizes
numThreadsPerBlock = 32
numBlocksx = math.ceil(C_n_cols/numThreadsPerBlock)
numBlocksy = math.ceil(C_n_rows/numThreadsPerBlock)

# Generate the matrices
A = cp.random.randn(A_n_rows, A_n_cols)
A = A.astype(value_type)

B = cp.random.randn(B_n_rows, B_n_cols)
B = B.astype(value_type)

C = cp.zeros([C_n_rows,C_n_cols])
C = C.astype(value_type)

# Call the CUDA kernel for matrix multiplication
our_kernel((numBlocksx, numBlocksy,1),(numThreadsPerBlock,numThreadsPerBlock, 1),(A, B, C, np.int32(A_n_rows), np.int32(A_n_cols), np.int32(B_n_cols)))

# Check the result by comparing it to the NumPy result
C_cupy = cp.dot(A,B)
if (np.allclose(C_cupy, C, 0.001, 0.001)):
  print("They are the same!")
else:
  print("Something must have gone wrong.:(")

They are the same!
